# 00_FiNER139-TagMerging
### Purpose: Mergin the original 139 different Labels into 10
<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 1. Import the FiNER-139 datset form Hugging Face 

In [1]:
from datasets import load_dataset
import pandas as pd

# Load the dataset from Hugging Face
dataset = load_dataset("nlpaueb/finer-139")

dataset
#dataset["train"][1]
label_names = dataset["train"].features["ner_tags"].feature.names
len(label_names)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 900384
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 112494
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 108378
    })
})

<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 2. Change number labels into text labels and print dataset structure (Dataset Dictionary with three elements)

In [2]:
# Function to convert numerical labels to names
def decode_ner_tags(example):
    example["ner_tags_str"] = [label_names[i] for i in example["ner_tags"]]
    return example


dataset_dict_named = {
    split: ds.map(decode_ner_tags)
    for split, ds in dataset.items()
}

dataset_dict_named

{'train': Dataset({
     features: ['id', 'tokens', 'ner_tags', 'ner_tags_str'],
     num_rows: 900384
 }),
 'validation': Dataset({
     features: ['id', 'tokens', 'ner_tags', 'ner_tags_str'],
     num_rows: 112494
 }),
 'test': Dataset({
     features: ['id', 'tokens', 'ner_tags', 'ner_tags_str'],
     num_rows: 108378
 })}

<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 3. Print one element of the Datset Dict and count length of shortes and longest row across all three splits

In [3]:
# Example: Output of a sample with label names
print(dataset_dict_named["train"][1])

{'id': 1, 'tokens': ['See', 'accompanying', 'notes', 'to', 'condensed', 'consolidated', 'financial', 'statements', '.', '3', 'Lennar', 'Corporation', 'and', 'Subsidiaries', 'Condensed', 'Consolidated', 'Statements', 'of', 'Operations', 'and', 'Comprehensive', 'Income', '(', 'Dollars', 'in', 'thousands', ',', 'except', 'per', 'share', 'amounts', ')', '(', 'unaudited', ')', 'See', 'accompanying', 'notes', 'to', 'condensed', 'consolidated', 'financial', 'statements', '.', '4', 'Lennar', 'Corporation', 'and', 'Subsidiaries', 'Condensed', 'Consolidated', 'Statements', 'of', 'Cash', 'Flows', '(', 'In', 'thousands', ')', '(', 'unaudited', ')', 'See', 'accompanying', 'notes', 'to', 'condensed', 'consolidated', 'financial', 'statements', '.', '5', 'Lennar', 'Corporation', 'and', 'Subsidiaries', 'Condensed', 'Consolidated', 'Statements', 'of', 'Cash', 'Flows', '(', 'In', 'thousands', ')', '(', 'unaudited', ')', 'See', 'accompanying', 'notes', 'to', 'condensed', 'consolidated', 'financial', 'stat

In [4]:
# Funktion zur Ermittlung der min/max Tokenlänge und deren Position im Split
def get_min_max_lengths(split):
    lengths = [len(example['tokens']) for example in dataset[split]]
    
    min_length = min(lengths)
    max_length = max(lengths)
    
    min_index = lengths.index(min_length)
    max_index = lengths.index(max_length)
    
    return {
        "split": split,
        "min_length": min_length,
        "min_index": min_index,
        "max_length": max_length,
        "max_index": max_index
    }

# Ergebnisse für alle Splits sammeln
results = [get_min_max_lengths(split) for split in dataset]

# In DataFrame zur Übersicht
import pandas as pd
df_results = pd.DataFrame(results)
print(df_results)


        split  min_length  min_index  max_length  max_index
0       train           2      68225        1260      49802
1  validation           2      12449        1157      32348
2        test           2       7921        1193      69243


<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 4. Manual mapping form the 139 tags to 10 tags

In [5]:
mapping = {
    "AccrualForEnvironmentalLossContingencies": "Contingency",
    "AcquiredFiniteLivedIntangibleAssetsWeightedAverageUsefulLife": "Assets",
    "AllocatedShareBasedCompensationExpense": "Compensation",
    "AmortizationOfFinancingCosts": "Debt",
    "AmortizationOfIntangibleAssets": "Assets",
    "AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount": "Equity",
    "AreaOfRealEstateProperty": "Assets",
    "AssetImpairmentCharges": "Assets",
    "BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued": "Acquisition",
    "BusinessAcquisitionPercentageOfVotingInterestsAcquired": "Acquisition",
    "BusinessCombinationAcquisitionRelatedCosts": "Acquisition",
    "BusinessCombinationConsiderationTransferred1": "Acquisition",
    "BusinessCombinationContingentConsiderationLiability": "Acquisition",
    "BusinessCombinationRecognizedIdentifiableAssetsAcquiredAndLiabilitiesAssumedIntangibleAssetsOtherThanGoodwill": "Assets",
    "BusinessCombinationRecognizedIdentifiableAssetsAcquiredAndLiabilitiesAssumedIntangibles": "Assets",
    "CapitalizedContractCostAmortization": "Revenue",
    "CashAndCashEquivalentsFairValueDisclosure": "Other",
    "ClassOfWarrantOrRightExercisePriceOfWarrantsOrRights1": "Equity",
    "CommonStockCapitalSharesReservedForFutureIssuance": "Equity",
    "CommonStockDividendsPerShareDeclared": "Equity",
    "CommonStockParOrStatedValuePerShare": "Equity",
    "CommonStockSharesAuthorized": "Equity",
    "CommonStockSharesOutstanding": "Equity",
    "ConcentrationRiskPercentage1": "Contingency",
    "ContractWithCustomerLiability": "Revenue",
    "ContractWithCustomerLiabilityRevenueRecognized": "Revenue",
    "CumulativeEffectOfNewAccountingPrincipleInPeriodOfAdoption": "Other",
    "DebtInstrumentBasisSpreadOnVariableRate1": "Debt",
    "DebtInstrumentCarryingAmount": "Debt",
    "DebtInstrumentConvertibleConversionPrice1": "Debt",
    "DebtInstrumentFaceAmount": "Debt",
    "DebtInstrumentFairValue": "Debt",
    "DebtInstrumentInterestRateEffectivePercentage": "Debt",
    "DebtInstrumentInterestRateStatedPercentage": "Debt",
    "DebtInstrumentMaturityDate": "Debt",
    "DebtInstrumentRedemptionPricePercentage": "Debt",
    "DebtInstrumentTerm": "Debt",
    "DebtInstrumentUnamortizedDiscount": "Debt",
    "DebtWeightedAverageInterestRate": "Debt",
    "DeferredFinanceCostsGross": "Debt",
    "DeferredFinanceCostsNet": "Debt",
    "DefinedBenefitPlanContributionsByEmployer": "Compensation",
    "DefinedContributionPlanCostRecognized": "Compensation",
    "Depreciation": "Assets",
    "DerivativeFixedInterestRate": "Debt",
    "DerivativeNotionalAmount": "Debt",
    "DisposalGroupIncludingDiscontinuedOperationConsideration": "Assets",
    "EffectiveIncomeTaxRateContinuingOperations": "Tax",
    "EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate": "Tax",
    "EmployeeServiceShareBasedCompensationNonvestedAwardsTotalCompensationCostNotYetRecognized": "Compensation",
    "EmployeeServiceShareBasedCompensationNonvestedAwardsTotalCompensationCostNotYetRecognizedPeriodForRecognition1": "Compensation",
    "EmployeeServiceShareBasedCompensationNonvestedAwardsTotalCompensationCostNotYetRecognizedShareBasedAwardsOtherThanOptions": "Compensation",
    "EmployeeServiceShareBasedCompensationTaxBenefitFromCompensationExpense": "Compensation",
    "EquityMethodInvestmentOwnershipPercentage": "Acquisition",
    "EquityMethodInvestments": "Acquisition",
    "FiniteLivedIntangibleAssetUsefulLife": "Assets",
    "GainsLossesOnExtinguishmentOfDebt": "Debt",
    "Goodwill": "Assets",
    "GoodwillImpairmentLoss": "Assets",
    "GuaranteeObligationsMaximumExposure": "Contingency",
    "IncomeLossFromEquityMethodInvestments": "Acquisition",
    "IncomeTaxExpenseBenefit": "Tax",
    "InterestExpense": "Debt",
    "InterestExpenseDebt": "Debt",
    "LeaseAndRentalExpense": "Lease",
    "LesseeOperatingLeaseRenewalTerm": "Lease",
    "LesseeOperatingLeaseTermOfContract": "Lease",
    "LettersOfCreditOutstandingAmount": "Debt",
    "LineOfCredit": "Debt",
    "LineOfCreditFacilityCommitmentFeePercentage": "Debt",
    "LineOfCreditFacilityCurrentBorrowingCapacity": "Debt",
    "LineOfCreditFacilityInterestRateAtPeriodEnd": "Debt",
    "LineOfCreditFacilityMaximumBorrowingCapacity": "Debt",
    "LineOfCreditFacilityRemainingBorrowingCapacity": "Debt",
    "LineOfCreditFacilityUnusedCapacityCommitmentFeePercentage": "Debt",
    "LongTermDebt": "Debt",
    "LongTermDebtFairValue": "Debt",
    "LossContingencyAccrualAtCarryingValue": "Contingency",
    "LossContingencyDamagesSoughtValue": "Contingency",
    "LossContingencyEstimateOfPossibleLoss": "Contingency",
    "LossContingencyPendingClaimsNumber": "Contingency",
    "MinorityInterestOwnershipPercentageByNoncontrollingOwners": "Other",
    "MinorityInterestOwnershipPercentageByParent": "Other",
    "NumberOfOperatingSegments": "Other",
    "NumberOfRealEstateProperties": "Assets",
    "NumberOfReportableSegments": "Other",
    "OperatingLeaseCost": "Lease",
    "OperatingLeaseExpense": "Lease",
    "OperatingLeaseLiability": "Lease",
    "OperatingLeasePayments": "Lease",
    "OperatingLeaseRightOfUseAsset": "Lease",
    "OperatingLeaseWeightedAverageDiscountRatePercent": "Lease",
    "OperatingLeaseWeightedAverageRemainingLeaseTerm1": "Lease",
    "OperatingLeasesRentExpenseNet": "Lease",
    "OperatingLossCarryforwards": "Tax",
    "PaymentsToAcquireBusinessesGross": "Acquisition",
    "PaymentsToAcquireBusinessesNetOfCashAcquired": "Acquisition",
    "PreferredStockDividendRatePercentage": "Equity",
    "PreferredStockSharesAuthorized": "Equity",
    "ProceedsFromIssuanceOfCommonStock": "Equity",
    "PropertyPlantAndEquipmentUsefulLife": "Assets",
    "PublicUtilitiesRequestedRateIncreaseDecreaseAmount": "Other",
    "RelatedPartyTransactionAmountsOfTransaction": "Other",
    "RelatedPartyTransactionExpensesFromTransactionsWithRelatedParty": "Other",
    "RepaymentsOfDebt": "Debt",
    "RestructuringAndRelatedCostExpectedCost1": "Contingency",
    "RestructuringCharges": "Contingency",
    "RevenueFromContractWithCustomerExcludingAssessedTax": "Revenue",
    "RevenueFromContractWithCustomerIncludingAssessedTax": "Revenue",
    "RevenueFromRelatedParties": "Revenue",
    "RevenueRemainingPerformanceObligation": "Revenue",
    "Revenues": "Revenue",
    "SaleOfStockNumberOfSharesIssuedInTransaction": "Equity",
    "SaleOfStockPricePerShare": "Equity",
    "ShareBasedCompensation": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardAwardVestingPeriod1": "Compensation",
    "SharebasedCompensationArrangementBySharebasedPaymentAwardAwardVestingRightsPercentage": "Compensation",
    "SharebasedCompensationArrangementBySharebasedPaymentAwardExpirationPeriod": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsGrantsInPeriod": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsGrantsInPeriodWeightedAverageGrantDateFairValue": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsNonvestedNumber": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsVestedInPeriodTotalFairValue": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardNumberOfSharesAuthorized": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardNumberOfSharesAvailableForGrant": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsExercisesInPeriodTotalIntrinsicValue": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsGrantsInPeriodGross": "Compensation",
    "ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsGrantsInPeriodWeightedAverageGrantDateFairValue": "Compensation",
    "SharePrice": "Equity",
    "StockIssuedDuringPeriodSharesNewIssues": "Equity",
    "StockRepurchaseProgramAuthorizedAmount1": "Equity",
    "StockRepurchaseProgramRemainingAuthorizedRepurchaseAmount1": "Equity",
    "StockRepurchasedAndRetiredDuringPeriodShares": "Equity",
    "StockRepurchasedDuringPeriodShares": "Equity",
    "SupplementalInformationForPropertyCasualtyInsuranceUnderwritersPriorYearClaimsAndClaimsAdjustmentExpense": "Assets",
    "TreasuryStockAcquiredAverageCostPerShare": "Equity",
    "TreasuryStockSharesAcquired": "Equity",
    "TreasuryStockValueAcquiredCostMethod": "Equity",
    "UnrecognizedTaxBenefits": "Tax",
    "UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate": "Tax",
}


<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 5. Function for Re-Mapping and applying on the Dataset Dict

In [6]:
# Function for Re-Mapping
def remap_ner_tags(example):
    def remap_label(label):
        if label == "O":
            return "O"
        # Trenne Prefix (B-/I-) und Kategorie
        prefix, _, tag = label.partition("-")
        new_tag = mapping.get(tag, tag)  # falls nicht im Mapping, original behalten
        return f"{prefix}-{new_tag}"
    
    example["ner_tags_str_mapped"] = [remap_label(label) for label in example["ner_tags_str"]]
    return example

# Apply on all splits
dataset_dict_remapped = {
    split: ds.map(remap_ner_tags)
    for split, ds in dataset_dict_named.items()
}

In [7]:
# Example: Displaying a sample with label names
print(dataset_dict_remapped["test"][7])

{'id': 1012885, 'tokens': ['The', 'effective', 'tax', 'rate', 'was', '47.6', 'percent', 'and', '13.4', 'percent', 'for', 'the', 'three', 'months', 'ended', 'March', '31', ',', '2020', 'and', '2019', ',', 'respectively', '.'], 'ner_tags': [0, 0, 0, 0, 0, 57, 0, 0, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ner_tags_str': ['O', 'O', 'O', 'O', 'O', 'B-EffectiveIncomeTaxRateContinuingOperations', 'O', 'O', 'B-EffectiveIncomeTaxRateContinuingOperations', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'ner_tags_str_mapped': ['O', 'O', 'O', 'O', 'O', 'B-Tax', 'O', 'O', 'B-Tax', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [8]:
# Function for removing the 'ner_tags' field
def remove_ner_tags(example):
    example.pop("ner_tags", None)
    example.pop("ner_tags_str", None)
    return example

# Apply on all splits
final_dataset = {
    split: ds.map(remove_ner_tags)
    for split, ds in dataset_dict_remapped.items()
}


In [9]:
# Example: Displaying a sample with label names
print(final_dataset["test"][7])

{'id': 1012885, 'tokens': ['The', 'effective', 'tax', 'rate', 'was', '47.6', 'percent', 'and', '13.4', 'percent', 'for', 'the', 'three', 'months', 'ended', 'March', '31', ',', '2020', 'and', '2019', ',', 'respectively', '.'], 'ner_tags_str_mapped': ['O', 'O', 'O', 'O', 'O', 'B-Tax', 'O', 'O', 'B-Tax', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [10]:
# In ein DataFrame umwandeln
df = pd.DataFrame({
    'token': final_dataset["test"][7]['tokens'],
    'ner_tag': final_dataset["test"][7]['ner_tags_str_mapped']
})

df

,token,ner_tag
0,The,O
1,effective,O
2,tax,O
3,rate,O
4,was,O
5,47.6,B-Tax
6,percent,O
7,and,O
8,13.4,B-Tax
9,percent,O


<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 6. Count occurences across al differents tags

In [11]:
import pandas as pd

# List of all labels of train-Split
all_labels = [label for sequence in final_dataset["train"]["ner_tags_str_mapped"] for label in sequence]

# Convert to DataFrame
label_df = pd.DataFrame({"label": all_labels})

# Count the number for tokens for every labels
label_counts = label_df["label"].value_counts()
print(label_counts)

label
O                 40508701
B-Debt               95162
B-Compensation       45433
B-Equity             32038
B-Assets             27150
B-Acquisition        20658
B-Lease              19048
B-Contingency        18647
B-Tax                17961
B-Revenue            17585
B-Other              14312
I-Debt                4841
I-Compensation        3215
I-Lease               1418
I-Assets              1240
I-Equity                76
I-Contingency            7
I-Acquisition            4
I-Other                  3
Name: count, dtype: int64


In [12]:
#label_names = final_dataset["train"].features["ner_tags_str_mapped"].unique()
label_names = set(label for row in final_dataset["train"]["ner_tags_str_mapped"] for label in row)
len(label_names)

19

In [13]:
final_dataset

{'train': Dataset({
     features: ['id', 'tokens', 'ner_tags_str_mapped'],
     num_rows: 900384
 }),
 'validation': Dataset({
     features: ['id', 'tokens', 'ner_tags_str_mapped'],
     num_rows: 112494
 }),
 'test': Dataset({
     features: ['id', 'tokens', 'ner_tags_str_mapped'],
     num_rows: 108378
 })}

<hr style="height:3px; width:100%; background-color:black; border:none; margin:auto;" />

## 7. Save new Dataset Dict as JSONL

In [14]:
# Save all splits as .jsonl
for split, ds in final_dataset.items():
    filename = f"data/finer-{split}.jsonl"
    ds.to_json(filename, orient="records", lines=True)
    print(f"{split} saved as: {filename}")


Creating json from Arrow format:   0%|          | 0/901 [00:00<?, ?ba/s]

train saved as: data/finer-train.jsonl


Creating json from Arrow format:   0%|          | 0/113 [00:00<?, ?ba/s]

validation saved as: data/finer-validation.jsonl


Creating json from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

test saved as: data/finer-test.jsonl
